In [1]:
import pandas as pd
import pickle
from langchain_community.vectorstores import chroma as Chroma
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_core.vectorstores import VectorStoreRetriever
from langchain_community.document_loaders import DataFrameLoader
from langchain.indexes import SQLRecordManager, index
from langchain_core.documents import Document
import json
from Text_preprocessing import Text_preprocessing
from sentence_transformers import SentenceTransformer
from chromadb.utils import embedding_functions 
from langchain.embeddings import SentenceTransformerEmbeddings

/home/mbhatti/miniconda3/envs/llama/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#FSD_1555
dataPath = "/home/mbhatti/mnt/d/LLM-repo1/models/langchain_implementation/fsd_1555_Japan_06_15.pkl"
# dataPath = "/home/mbhatti/mnt/d/LLM-repo1/models/langchain_implementation/fsd_1555_0601_06_15.pkl"
dateFrom = "2023-06-01 06:00:00+00:00" 
dateTo = "2023-06-01 13:00:00+00:00" 

# Loading pandas dataframe from picke file
with open(dataPath, 'rb') as f:
    data = pickle.load(f)

df = pd.DataFrame(data)
df['date'] = pd.to_datetime(df['date'])
# df = df.drop(columns=['id','tag_class', 'source', 'lang', 'urls','locations'])

#Get data between thresholds
threshold_datetime_lower = pd.to_datetime(dateFrom)
threshold_datetime_upper = pd.to_datetime(dateTo)
df = df[df['date'] >= threshold_datetime_lower]
df = df[df['date'] <= threshold_datetime_upper]

#Remove duplicates
# df_new  = df.drop_duplicates(subset=["text"], keep=False)

#Covert date to string
df['date'] = df['date'].astype(str)
data = df

In [3]:
pd.set_option('display.max_colwidth', None)
data

,date,text
6817,2023-06-01 12:59:40+00:00,美濃加茂は川あり山あり、平地あり。 自分の子供の頃は、太田地区の水害は毎年の事でしたが。 今は、だいぶ整理されたとは言え、局地的に降るとどうなるか読めませんよね。 大事に至らないのを祈るばかりです
6818,2023-06-01 12:58:00+00:00,線状降水帯が発生した場合は、局地的に雨量が増えるおそれがあります。 西日本から東日本の広い範囲で、土砂災害、低い土地の浸水、河川の増水や氾濫に警戒してください。
6819,2023-06-01 12:57:26+00:00,◉台風2号接近のため注意喚起 気象庁の予報によると、明日6/2-6/3は台風接近に伴い、雨風が強まる可能性があります。明日、横浜は開港記念日。大変残念ですが安全が第一。都筑区の浸水ハザードマップ。川沿いエリアは特に注意してください。 #横浜市 の他エリアはこちらから
6820,2023-06-01 12:57:16+00:00,【あす2日 「災害級の大雨」の恐れ 線状降水帯発生の可能性も 避難の心構えを】tenkijp とは言うが台風の勢力は衰え始め、東京は関西程ではないのでは？と希望的観測。 全ては梅雨前線次第
6821,2023-06-01 12:57:02+00:00,あれよな3.11ぐらいまではさ ネタ無くて ニュースも みーーーーーーんな 『藤井名人誕生』みたいな!!そう言うネタばかりだった 思うと3.11以降・・色々起き過ぎたわな;;思えばいがみ合いもここ等辺から原発・燃料そして熊本に洪水とコロナとロシアと・・そろそろ平穏期入って欲しいな。 昔はさ・・・
...,...,...
7372,2023-06-01 06:09:02+00:00,台風２号の影響で3日にかけて東海3県は大雨の恐れ 土砂災害や浸水に警戒(メ〜テレ（名古屋テレビ）) #Yahooニュース
7373,2023-06-01 06:08:47+00:00,まさかの2か月連続台風… それが無ければフローラ姫さんか、あやママさんで決まってたかな？と、思うけど、それも桃鉄。 でも、珍しいも。 水俣の畑に被害は無く… ほっとしました。
7374,2023-06-01 06:05:16+00:00,ハザードマップを確認 台風2号の北上にともなって、 日本の南にある前線の活動が活発になる見込みで、 東海3県では、今夜から3日にかけて雷をともなった非常に激しい雨が降るところがある見込みです。 自分が住む地域の災害の危険性を事前に確認し、避難の場所や方法を確認しておきましょう。
7375,2023-06-01 06:01:47+00:00,今後36時間で、300mmを超えるような予想のところもあります。この図はあくまでコンピュータの予想で、局地的にはそれよりも多くなる可能性もあります。 南西諸島から東日本の広い範囲で、土砂災害、低い土地の浸水、河川の増水や氾濫に警戒してください。


In [2]:
##FSD_1777
dataPath = "/home/mbhatti/mnt/d/LLM-repo1/models/langchain_implementation/FSD1777_Oct23.json"
dateFrom = "2023-10-19T09:00:00+00:00" #2023-10-19T18:58:41Z for 200 tweets
dateTo = "2023-10-19T18:00:00+00:00"

"""Load relevant fields of flood tags api json response"""
def json_dataloader(dataPath = dataPath, dateFrom = dateFrom, dateTo = dateTo):
    # Load json and extract relevant records in pandas df
    with open(dataPath, 'r') as json_file:
        response_dict = json.load(json_file)

    # Convert to pandas df    
    pd.set_option('display.max_colwidth', None)
    df = pd.DataFrame(response_dict)
    df['date'] = pd.to_datetime(df['date'])
    df = df.drop(columns=['id','tag_class', 'source', 'lang', 'urls','locations'])

    #Get data between thresholds
    threshold_datetime_lower = pd.to_datetime(dateFrom)
    threshold_datetime_upper = pd.to_datetime(dateTo)
    df = df[df['date'] >= threshold_datetime_lower]
    df = df[df['date'] <= threshold_datetime_upper]

    #Remove duplicates
    df  = df.drop_duplicates(subset=["text"], keep=False)
    #Pre-process
    preprocess = Text_preprocessing(df)
    df = preprocess.preprocess()
    #Covert date to string
    df['date'] = df['date'].astype(str)
    return df

# Load the data from source
data = json_dataloader()


In [ ]:
def json_dataloader(dataPath = dataPath, dateFrom = dateFrom, dateTo = dateTo):
    # Load json and extract relevant records in pandas df
    with open(dataPath, 'r') as json_file:
        response_dict = json.load(json_file)

    # Convert to pandas df    
    pd.set_option('display.max_colwidth', None)
    df = pd.DataFrame(response_dict)
    df['date'] = pd.to_datetime(df['date'])
    df = df.drop(columns=['id','tag_class', 'source', 'lang', 'urls','locations'])

    #Get data between thresholds
    threshold_datetime_lower = pd.to_datetime(dateFrom)
    threshold_datetime_upper = pd.to_datetime(dateTo)
    df = df[df['date'] >= threshold_datetime_lower]
    df = df[df['date'] <= threshold_datetime_upper]

    #Remove duplicates
    df  = df.drop_duplicates(subset=["text"], keep=True)
    #Pre-process
    preprocess = Text_preprocessing(df)
    df = preprocess.preprocess()
    #Covert date to string
    df['date'] = df['date'].astype(str)
    return df

# Load the data from source
data = json_dataloader()


In [20]:
from langchain.embeddings import SentenceTransformerEmbeddings
from FlagEmbedding import FlagReranker
from pydantic.v1 import Field
from typing import List, Dict, Any, Mapping
from langchain_core.vectorstores import VectorStoreRetriever
from langchain_core.documents import Document
#Cross encoder re rank
class Custom_ja_Retriever(VectorStoreRetriever):
    """Implements re ranking of retriever output using cross encoder"""
    vectorstore: VectorStoreRetriever
    search_type: str = "similarity"
    search_kwargs: dict = Field(default_factory=dict)

    def get_relevant_documents(self, query: str) -> List[Document]:
        """Extract relevant records using japanese embedding model"""

        #Translate query to japanese 
        # query_ja = llmModels.bulk_translation(query, source_lan = "en_XX", output_lan= "ja_XX")
        # query = query_ja[0]

        #TRANSLATION -------------------------------------------------------------------
        #Retrieve japanese records
        docs = self.vectorstore.get_relevant_documents(query=query)

        dataPath = "/home/mbhatti/mnt/d/LLM-repo1/models/langchain_implementation/fsd_1555_0601_06_15.pkl"
        dateFrom = "2023-06-01 06:00:00+00:00" 
        dateTo = "2023-06-01 13:00:00+00:00" 

        # Loading pandas dataframe from picke file
        with open(dataPath, 'rb') as f:
            data_eng = pickle.load(f)

        df = pd.DataFrame(data_eng)
        df['date'] = pd.to_datetime(df['date'])
        # df = df.drop(columns=['id','tag_class', 'source', 'lang', 'urls','locations'])

        #Get data between thresholds
        threshold_datetime_lower = pd.to_datetime(dateFrom)
        threshold_datetime_upper = pd.to_datetime(dateTo)
        df = df[df['date'] >= threshold_datetime_lower]
        df = df[df['date'] <= threshold_datetime_upper]

        #Remove duplicates
        # df_new  = df.drop_duplicates(subset=["text"], keep=False)

        #Covert date to string
        df['date'] = df['date'].astype(str)
        data_eng = df

        #Store japanese tweets in an array 
        ja_tweets = []
        for doc in docs:
            ja_tweets.append(doc.page_content)

        #Get index of these extracted tweets
        indexes = []
        for tweet in ja_tweets:
            indexes.append(data[data['text'] ==tweet].index[0])

                #Extract english from exact location
        docs_en = data_eng.loc[indexes].text.to_list()

        #Chnage text to english 
        ind = 0
        for doc in docs:
            doc.page_content = docs_en[ind]
            ind = ind+1


        # Cross encoder re ranking ------------------------------------------
        #For japanese query translation
        query = "Which areas are subject to flood warnings due to heavy rain?"
        cross_encoder = FlagReranker('BAAI/bge-reranker-v2-m3', use_fp16=True)

        pairs = []
        for doc in docs:
            pairs.append([query, doc.page_content])
        scores = cross_encoder.compute_score(pairs)

        i = 0
        #Add scores to document
        for doc in docs:
            doc.metadata["cross-encoder_score"] = scores[i]
            i = i+1
        #Sort documents according to score
        sorted_docs = sorted(docs, key=lambda doc: doc.metadata.get('cross-encoder_score'), reverse=True)

        #Remove 20 worst performing docs from the list 
        sorted_docs = sorted_docs[:k_cross]

        # #Re order according to long text re-order (Important context in start and end)
        # reordering = LongContextReorder()
        # reordered_docs = reordering.transform_documents(sorted_docs)

        #Remove cross encoder score so re ordered context is back to its orignal form
        for doc in sorted_docs:
            doc.metadata.pop("cross-encoder_score")

        #Change updated docs back to docs
        docs = sorted_docs

        ##Extract japanese tweet texts -----------
        # tweets_JA = []
        # for doc in docs:
        #     tweets_JA.append(doc.page_content)

        # #Translate text feild to english
        # # docs_en = llmModels.bulk_translation(tweets_JA)

        # #Chnage text to english 
        # ind = 0
        # for doc in docs:
        #     doc.page_content = docs_en[ind]
        #     ind = ind+1

        return docs

In [3]:
def bgeEmbeddings():
    model_name = "BAAI/bge-large-en-v1.5"
    # model_name = "BAAI/bge-m3"
    model_kwargs = {'device': 'cuda'}
    encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
    model = HuggingFaceBgeEmbeddings(
        model_name=model_name,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )
    return model

In [4]:
embeddings = bgeEmbeddings()

# embeddings = SentenceTransformer('pkshatech/GLuCoSE-base-ja')
# embeddings = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="pkshatech/GLuCoSE-base-ja")
# embeddings = SentenceTransformerEmbeddings(model_name="pkshatech/GLuCoSE-base-ja")

documents = []
loader = DataFrameLoader(data, page_content_column="text")
documents.extend(loader.load())
#Create a vector store
# db = Chroma.Chroma("Langchain collection",embeddings)
db = Chroma.Chroma.from_documents(documents,
                                  embeddings)
if db._client.list_collections() != None:

  for collection in db._client.list_collections():
    ids = collection.get()['ids']
    print('REMOVE %s document(s) from %s collection' % (str(len(ids)), collection.name))
    if len(ids): collection.delete(ids)

db = Chroma.Chroma.from_documents(documents,
                                  embeddings)

REMOVE 555 document(s) from langchain collection


In [5]:
# Get all embeddings
len(db._collection.get()['ids'])


555

In [17]:
retriever = db.as_retriever(search_kwargs={'k': 20})
query = """Whihch locations have received evacuation orders due to Storm Babet?"""
# k_cross = 50
# retriever = Custom_ja_Retriever(vectorstore=db.as_retriever(search_kwargs={'k': 100}))

# query = """Which locations are likely to receive flood warnings due to heavy rain?"""
docs = retriever.get_relevant_documents(query=query)

len(docs)


20

In [9]:
print(docs)

[Document(page_content='#StormBabet We are getting hit now Aberdeenshire! Some houses evacuated, flooding, bridges shut, roads flooded, flood gates shut, trees down, power outages. Look after each other, neighbours and animals. Stay safe!', metadata={'date': '2023-10-19 15:13:40+00:00'}), Document(page_content='Wild up here #thursdaymorning in #Aberdeen #aberdeenshire as #StormBabet sweeps the east, north with gale force winds &amp; heavy rainst now. Some schools, roads closed so travel safely everyone as #flooding bad in many riverside area(: #StormHour', metadata={'date': '2023-10-19 09:56:17+00:00'}), Document(page_content='STAY CLEAR OF FLOODED ROADS. Flooded roads often obscure potential hazards DO NOT DRIVE THROUGH ROAD CLOSURES Stay safe, turn around and seek an alternative route if travel is unavoidable! #Aberdeenshire #StormBabet', metadata={'date': '2023-10-19 10:45:27+00:00'}), Document(page_content='Scotland is preparing for flooding as #StormBabet approaches. Stay up to da

In [9]:
#FSD_1555 --- To track English tweets wothout translation 
dataPath = "/home/mbhatti/mnt/d/LLM-repo1/models/langchain_implementation/fsd_1555_0601_06_15.pkl"
dateFrom = "2023-06-01 06:00:00+00:00" 
dateTo = "2023-06-01 13:00:00+00:00" 

# Loading pandas dataframe from picke file
with open(dataPath, 'rb') as f:
    data_eng = pickle.load(f)

df = pd.DataFrame(data_eng)
df['date'] = pd.to_datetime(df['date'])
# df = df.drop(columns=['id','tag_class', 'source', 'lang', 'urls','locations'])

#Get data between thresholds
threshold_datetime_lower = pd.to_datetime(dateFrom)
threshold_datetime_upper = pd.to_datetime(dateTo)
df = df[df['date'] >= threshold_datetime_lower]
df = df[df['date'] <= threshold_datetime_upper]

#Remove duplicates
# df_new  = df.drop_duplicates(subset=["text"], keep=False)

#Covert date to string
df['date'] = df['date'].astype(str)
data_eng = df

#Store japanese tweets in an array 
ja_tweets = []
for doc in docs:
    ja_tweets.append(doc.page_content)

#Get index of these extracted tweets
indexes = []
for tweet in ja_tweets:
    indexes.append(data[data['text'] ==tweet].index[0])

#Search for indexes over the english data
print(len(indexes))
print(data_eng.loc[indexes].text.to_list())
# with open("Output.txt", "w") as text_file:
#     text_file.write(data_eng.loc[indexes].text.to_string())

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
ja_tweets[0]


In [ ]:
eng_string = ['String 1', 'String 2', 'String 3', 'String 4', 'String 5']

#Extract japanese tweet texts
tweets_JA = []
ind = 0
for doc in docs:
    doc.page_content = eng_string[ind]
    ind = ind+1

In [ ]:
docs

In [ ]:
#Testing UAE embeddings
from scipy import spatial
input = "Which locations have recieved an evacuation order?"
query = embeddings.embed_query (input)
docs = retriever.get_relevant_documents(query=input)

doc_vecs = []
for doc in docs:
    doc_vecs.append(embeddings.embed_query (doc.page_content))



In [ ]:
cosine_sim = []
for dv in doc_vecs:
    cosine_sim.append(spatial.distance.cosine(query, dv))

    # print(len(dv))

In [ ]:
query = """Aberdeenshire"""
# [d[1] for d in db.similarity_search_with_score(query, k=300)]
outp =  db.similarity_search_with_score(query, k=300)

In [ ]:
i = 0
for element in cosine_sim:
    if element < 0.58:
        print(i)
    i = i+1

In [ ]:
print(cosine_sim)

In [ ]:
#New embeddging model
from transformers import AutoModel, AutoTokenizer

input_texts = [
    "what is the capital of China?",
    "how to implement quick sort in python?",
    "Beijing",
    "sorting algorithms"
]

model_path = 'Alibaba-NLP/gte-large-en-v1.5'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModel.from_pretrained(model_path, trust_remote_code=True)

# Tokenize the input texts
batch_dict = tokenizer(input_texts, max_length=8192, padding=True, truncation=True, return_tensors='pt')

outputs = model(**batch_dict)
embeddings = outputs.last_hidden_state[:, 0]

In [ ]:
e = embeddings.tolist()


In [ ]:
from sentence_transformers import SentenceTransformer
from langchain_experimental.text_splitter import SemanticChunker
from typing import List


class MyEmbeddings:
    def __init__(self):
        self.model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        return [self.model.encode(t).tolist() for t in texts]


embeddings = MyEmbeddings()

# splitter = SemanticChunker(embeddings)

In [ ]:
# creating custom embeddings with non-default embedding model

from chromadb import Documents, EmbeddingFunction, Embeddings

class MyEmbeddingFunction(EmbeddingFunction):
    def __call__(self, input: Documents) -> Embeddings:
        ##
        #New embeddging model
        from transformers import AutoModel, AutoTokenizer


        model_path = 'Alibaba-NLP/gte-large-en-v1.5'
        tokenizer = AutoTokenizer.from_pretrained(model_path)
        model = AutoModel.from_pretrained(model_path, trust_remote_code=True)

        # Tokenize the input texts
        batch_dict = tokenizer(input, max_length=8192, padding=True, truncation=True, return_tensors='pt')

        outputs = model(**batch_dict)
        embeddings = outputs.last_hidden_state[:, 0]
                
        return embeddings.tolist()
        
        
        
custom_embeddings=MyEmbeddingFunction()


In [ ]:
# embeddings = model

documents = []
loader = DataFrameLoader(data, page_content_column="text")
documents.extend(loader.load())
print(documents)

# db = Chroma.Chroma.from_documents(documents,
#                                   custom_embeddings,
#                                   collection_metadata={"hnsw:space": "cosine"})
